In [1]:
import numpy as np
import scipy.sparse as sparse

# Initialization

In [2]:
def record_output(file_name: str, vars: list):
    """Writes/Appends the data given into the file address provided as comma separated values.

    Args:
        file_name (str): name of the file
        vars (list): list of the variables that their values will be written in the file
    """
    f = open(file_name, 'a')
    t = ','.join(np.array(vars).astype(str)) + '\n'
    f.write(t)
    f.close()

$$\mathbf{A}_{i j} = \underbrace{\overbrace{\mathbf{I} \otimes \ldots \otimes\mathbf{I}}^{N_j i + j\ \text{operators}} \otimes \sigma^z \otimes \overbrace{\mathbf{I} \otimes \ldots \otimes\mathbf{I}}^{N_j N_i - N_j i - j - 1\ \text{operators}}}_{N_i N_j\ \text{operators}}, \quad i,j = 0, 1, 2, \ldots$$

In [3]:
def n_identities(n: int) -> (sparse._dia.dia_matrix | int):
    """Returns a matrix made of the tensor product of n identity matrices.

    Args:
        n (int): number of the identity matrices

    Returns:
        (dia_matrix | int): an identity matrix of shape (2**n, 2**n)
    """
    return sparse.identity(2**n) if (n > 0) else 1

In [4]:
def A_operators(N_i: int, N_j: int) -> np.ndarray:
    """Return the A operators of the problem.

    Args:
        N_i (int): number of rows
        N_j (int): number of columns

    Returns:
        numpy.ndarray: an array of shape (N_i, N_j)
    """
    pauli_z = sparse.diags_array([1, -1])
    # A list to store the operators A
    A_ops = np.empty((N_i,N_j), dtype=sparse._coo.coo_matrix)

    for i in range(N_i):
        for j in range(N_j):
            A_ij = sparse.kron(sparse.kron(n_identities((N_j*i)+j),
                                           pauli_z),
                               n_identities((N_j*N_i)-(N_j*i)-j-1))
            A_ops[i,j] = A_ij

    return A_ops

$$\mathbf{H} = \epsilon \sum_{i, j = 0}^{N_i - 1, N_j - 1} \mathbf{A}_{i j} + \frac{1}{2} J \sum_{i, j = 0}^{N_i - 1, N_j - 1} \mathbf{A}_{i j} (\mathbf{A}_{i + 1, j} + \mathbf{A}_{i - 1, j} + \mathbf{A}_{i, j + 1} + \mathbf{A}_{i, j - 1})$$

In [5]:
def hamiltonian(N_i: int, N_j: int, A_ops: np.ndarray, epsilon: float, J: float, BC_type: str) -> sparse._csr.csr_matrix:
    """Returns the Hamiltonian of the system.

    Args:
        N_i (int): number of rows
        N_j (int): number of columns
        A_ops (numpy.ndarray): an array of A_{ij} operators
        epsilon (float): the magnetic field exerted on the system
        J (float): exchange energy
        BC_type (str): type of the boundary conditions:
            'PBC' (Periodic Boundary Condition),
            'APBC' (Anti-Periodic Boundary Condition).

    Raises:
        Exception:
            BC_type is not chosen correctly.
            the neighboring site on the boundaries of the lattice was not found (or incorrect index).

    Returns:
        csr_matrix: an sparse matrix representing the Hamiltonian of shape (2**(N_i*N_j), 2**(N_i*N_j))
    """
    # Creating the Hamiltonian
    H = sparse.dia_matrix((2**(N_i*N_j), 2**(N_i*N_j)), dtype=float)
    # Choosing the type of the BC
    match BC_type:
        case 'PBC':
            for i in range(N_i):
                for j in range(N_j):
                    H += epsilon * A_ops[i,j]
                    # Neighbors on the same row
                    for index in [j+1, j-1]:
                        # If index is (i, -1), there will be no problem
                        try:
                            H += 0.5 * J * (A_ops[i,j] @ A_ops[i, index])
                        # IndexError means that the index is out of bound,
                        # so index 0 should be used.
                        except IndexError:
                            try:
                                H += 0.5 * J * (A_ops[i,j] @ A_ops[i, 0])
                            except:
                                raise Exception('failed to find element \
                                ({:.0f},{:.0f}) in A_ops'.format(i, index))
                    # Neighbors on the same column
                    for index in [i+1, i-1]:
                        # If index is (-1, j), there will be no problem
                        try:
                            H += 0.5 * J * (A_ops[i,j] @ A_ops[index, j])
                        # IndexError means that the index is out of bound,
                        # so index 0 should be used.
                        except IndexError:
                            try:
                                H += 0.5 * J * (A_ops[i,j] @ A_ops[0, j])
                            except:
                                raise Exception('failed to find element \
                                ({:.0f},{:.0f}) in A_ops'.format(index, j))

        case 'APBC':
            for i in range(N_i):
                for j in range(N_j):
                    H += epsilon * A_ops[i,j]
                    # Neighbors on the same row
                    for index in [j+1, j-1]:
                        # If index is (i, -1), there will be no problem
                        try:
                            if index == -1:
                                H += 0.5 * J * (A_ops[i,j] @ (-A_ops[i, -1]))
                            else:
                                H += 0.5 * J * (A_ops[i,j] @ A_ops[i, index])
                        # IndexError means that the index is out of bound,
                        # so index 0 should be used.
                        except IndexError:
                            try:
                                H += 0.5 * J * (A_ops[i,j] @ (-A_ops[i, 0]))
                            except:
                                raise Exception('failed to find element \
                                ({:.0f},{:.0f}) in A_ops'.format(i, index))
                    # Neighbors on the same column
                    for index in [i+1, i-1]:
                        # If index is (-1, j), there will be no problem
                        try:
                            if index == -1:
                                H += 0.5 * J * (A_ops[i,j] @ (-A_ops[-1, j]))
                            else:
                                H += 0.5 * J * (A_ops[i,j] @ A_ops[index, j])
                        # IndexError means that the index is out of bound,
                        # so index 0 should be used.
                        except IndexError:
                            try:
                                H += 0.5 * J * (A_ops[i,j] @ (-A_ops[0, j]))
                            except:
                                raise Exception('failed to find element \
                                ({:.0f},{:.0f}) in A_ops'.format(index, j))
        case default:
            raise Exception('BC_type is not correctly defined')
    return H

In [6]:
def eigens(H_op: sparse.csr_matrix) -> tuple:
    """Returns half of the eigenvalues, the least eigenvalue (E_0) and its corresponding eigenvector.

    Args:
        H_op (csr_matrix): Hamiltonian of the system

    Returns:
        tuple: eigenvalues (ndarray), the least eigenvalue E_0 (float), eigenvector (ndarray)
    """
    # Getting the smallest eigenvalues and their corresponding eigenvectors
    # k=H_op.shape[0]//2 returns half of the eigenvalues of the system
    # sparse.linalg.eigs(H_op, k=H_op.shape[0]//2, which='SR') could be used but the results were inconsistent.
    eig_vals, eig_vecs = np.linalg.eig(H_op.todense())
    # Turning column presentation of eig_vecs into row presentation
    eig_vecs = eig_vecs.T
    # Removing unnecessary computed values
    eig_vecs[abs(eig_vecs) < 1e-10] = 0
    eig_vals = np.round(eig_vals, decimals=10)
    # The lowest energy
    E_0 = np.real(np.min(eig_vals))
    # The state with the lowest energy
    # Turning to column presentation
    psi_0 = eig_vecs[eig_vals == E_0].T
    # Checking the dimensions of psi_0
    # n case the degeneracy have given more than one eigenstates
    _, psi_0_col = psi_0.shape
    if psi_0_col > 1:
        # Setting the first state as the ground state
        psi_0 = psi_0[:,0].reshape(-1, 1)
    return eig_vals, E_0, psi_0

$$\mathbf{M} = \sum_{i, j = 1}^{N} \mathbf{A}_{i j}, \quad M_0 = \left< \psi_0 \right| \mathbf{M} \left| \psi_0 \right>$$

In [7]:
def magnetization_0(A_ops, psi_0: np.ndarray) -> float:
    """Returns the magnetization of the ground state at T=0.

    Args:
        A_ops (ndarray): an array of the A operators of the system
        psi_0 (ndarray): column eigenvector of the ground state

    Returns:
        float: magnetization
    """
    # A_ops.sum() is the magnetization operator
    M = psi_0.conjugate().T @ A_ops.sum() @ psi_0
    # Retrieving tne value from the M array
    if M.shape == (1, 1):
        M = np.real(M[0,0])

    return M

$$Z = \sum_n e^{-E_n / k_B T}$$

In [8]:
def partition(eig_vals: np.ndarray, k_B: float, T: float) -> float:
    """Returns the value of the partition function of a system.

    Args:
        eig_vals (ndarray): an array of eigenvalues
        k_B (float): Boltzman's constant
        T (float): temperature

    Returns:
        float: numerical value of the partition function
    """
    Z = 0
    for energy in eig_vals:
        Z += np.exp(-np.real(energy) / (k_B * T))

    return Z

$$F = - k_B T \ln Z = E - T S$$

In [9]:
def helmholtz(k_B: float, T: float, Z: float) -> float:
    """Returns the Helmholtz energy.

    Args:
        k_B (float): Boltzman's constant
        T (float): temperature
        Z (float): value of the partition function

    Returns:
        float: Helmholtz energy
    """
    return -k_B * T * np.log(Z)

$$\frac{\partial \ln Z}{\partial T} = \lim_{\delta T \rightarrow 0} \frac{\ln Z(T + \delta T) - \ln Z(T)}{\delta T}$$

In [10]:
def d_ln_Z_dT(eig_vals: np.ndarray, k_B: float, T: float, delta_T: float) -> float:
    """Returns the numerical value of (d/dT)ln(Z).

    Args:
        eig_vals (ndarray): eigenvalues (energy levels) of the system
        k_B (float): Boltzman's constant
        T (float): temperature
        delta_T (float): differentiation step

    Returns:
        float: derivative of ln(Z): (d/dT)ln(Z)
    """
    return (np.log(partition(eig_vals=eig_vals, k_B=k_B, T=T+delta_T)) 
            - np.log(partition(eig_vals=eig_vals, k_B=k_B, T=T))) / delta_T

$$\frac{\partial^2 \ln Z}{\partial^2 T} = \lim_{\delta T \rightarrow 0} \frac{\frac{\ln Z(T + 2\delta T) - \ln Z(T + \delta T)}{\delta T} - \frac{\ln Z(T + \delta T) - \ln Z(T)}{\delta T}}{\delta T}$$

In [11]:
def d2_ln_Z_d2T(eig_vals: np.ndarray, k_B: float, T: float, delta_T: float) -> float:
    """Returns the numerical value of (d**2/dT**2)ln(Z).

    Args:
        eig_vals (ndarray): eigenvalues (energy levels) of the system
        k_B (float): Boltzman's constant
        T (float): temperature
        delta_T (float): differentiation step

    Returns:
        float: derivative of ln(Z): (d**2/dT**2)ln(Z)
    """
    return (d_ln_Z_dT(eig_vals=eig_vals, k_B=k_B, T=T+delta_T, delta_T=delta_T) 
            - d_ln_Z_dT(eig_vals=eig_vals, k_B=k_B, T=T, delta_T=delta_T)) / delta_T

$$C_V = k_B T \left( 2 \left. \frac{\partial \ln Z}{\partial T} \right|_V + T \left. \frac{\partial^2 \ln Z}{\partial T^2} \right|_V \right)$$

In [12]:
def heat_cap_v(eig_vals: np.ndarray, k_B: float,
               T: float, delta_T: float) -> float:
    """Returns the heat capacity of the system.

    Args:
        eig_vals (ndarray): eigenvalues (energy levels) of the system
        k_B (float): Boltzman's constant
        T (float): temperature
        delta_T (float): differentiation step

    Returns:
        float: heat capacity of the system at the temperature T
    """
    return k_B * T * (2 * d_ln_Z_dT(eig_vals=eig_vals, k_B=k_B, T=T, delta_T=delta_T) + 
                      T * d2_ln_Z_d2T(eig_vals=eig_vals, k_B=k_B, T=T, delta_T=delta_T))

$$m = - \left. \frac{\partial F}{\partial \epsilon} \right|_T = \lim_{\delta \epsilon \rightarrow 0} \frac{F_\epsilon - F_{\epsilon + \delta \epsilon}}{\delta \epsilon}$$

In [42]:
def magnetization_T(N_i: int, N_j: int, k_B: float, BC_type: str,
                    epsilon: float, delta_epsilon: float,
                    J: float, T: float) -> float:
    """Returns the magnetization of the system at temperature T.

    Args:
        N_i (int): number of rows
        N_j (int): number of columns
        k_B (float): Boltzman's constant
        BC_type (str): type of the boundary conditions:
            'PBC' (Periodic Boundary Condition),
            'APBC' (Anti-Periodic Boundary Condition).
        epsilon (float): intensity of the magnetic field
        delta_epsilon (float): differentiation step
        J (float): exchange energy
        T (float): temperature

    Returns:
        float: magnetization
    """
    H_1 = hamiltonian(N_i=N_i, N_j=N_j,
                      A_ops=A_operators(N_i=N_i, N_j=N_j),
                      BC_type=BC_type, epsilon=epsilon, J=J)
    H_2 = hamiltonian(N_i=N_i, N_j=N_j,
                      A_ops=A_operators(N_i=N_i, N_j=N_j),
                      BC_type=BC_type, epsilon=epsilon+delta_epsilon, J=J)
    eig_vals_1, _, _ = eigens(H_op=H_1)
    eig_vals_2, _, _ = eigens(H_op=H_2)
    # Passing eigenvalues to the partition function and calculating F
    F_1 = helmholtz(k_B=k_B, T=T, Z=partition(eig_vals=eig_vals_1, k_B=k_B, T=T))
    F_2 = helmholtz(k_B=k_B, T=T, Z=partition(eig_vals=eig_vals_2, k_B=k_B, T=T))
    
    return (F_1 - F_2) / delta_epsilon

# Computation

## Parameters

In [35]:
# Number of rows and columns
N_i, N_j        = 3, 3
# Magnetic field
epsilon         = 0
# Exchange energy
J               = -1
# Temperature
T               = 0
# Type of the boundary condition
BC_type         = 'PBC'
# Differentiation steps
delta_epsilon   = 0.1
delta_T         = 1e-5
# Boltzman's constant
k_B             = 1

## T = 0

In [36]:
A_ops                   = A_operators(N_i=N_i, N_j=N_j)
H                       = hamiltonian(N_i=N_i, N_j=N_j, A_ops=A_ops, BC_type=BC_type, epsilon=epsilon, J=J)
eig_vals, E_0, psi_0    = eigens(H_op=H)
M_0                     = magnetization_0(A_ops=A_ops, psi_0=psi_0)
Z                       = partition(eig_vals=eig_vals, k_B=k_B, T=T)
F                       = helmholtz(k_B=k_B, T=T, Z=Z)
C_v                     = heat_cap_v(eig_vals=eig_vals, k_B=k_B, T=T, delta_T=delta_T)

print('E_0 =\n\t', E_0)
print('M_0 =\n\t', M_0)
print('Z =\n\t', Z)
print('F =\n\t', F)
print('C_V =\n\t', C_v)

E_0 =
	 -18.0
M_0 =
	 9.0
Z =
	 inf
F =
	 nan
C_V =
	 nan


/tmp/ipykernel_20814/143915578.py:14: RuntimeWarning: divide by zero encountered in scalar divide
  Z += np.exp(-np.real(energy) / (k_B * T))
/tmp/ipykernel_20814/2466187856.py:12: RuntimeWarning: invalid value encountered in scalar multiply
  return -k_B * T * np.log(Z)
/tmp/ipykernel_20814/143915578.py:14: RuntimeWarning: overflow encountered in exp
  Z += np.exp(-np.real(energy) / (k_B * T))
/tmp/ipykernel_20814/3190315398.py:13: RuntimeWarning: invalid value encountered in scalar subtract
  return (np.log(partition(eig_vals=eig_vals, k_B=k_B, T=T+delta_T))


## T > 0

In [43]:
A_ops = A_operators(N_i=N_i, N_j=N_j)
Ts = [0, 0.01, 0.1, 0.5, 1, 10]
for epsilon_ in [0]:
    for T_ in Ts:
        eig_vals, E_0, psi_0    = eigens(H_op=hamiltonian(N_i=N_i, N_j=N_j,
                                                          A_ops=A_ops, BC_type=BC_type,
                                                          epsilon=epsilon_, J=J))
        M_0                     = magnetization_0(A_ops=A_ops, psi_0=psi_0)
        M_T                     = magnetization_T(N_i=N_i, N_j=N_j, k_B=k_B, BC_type=BC_type,
                                                  epsilon=epsilon_, delta_epsilon=delta_epsilon,
                                                  J=J, T=T_)

        print('M(T=' + str(T_) +') =\n\t' + str(M_T))

/tmp/ipykernel_20814/143915578.py:14: RuntimeWarning: divide by zero encountered in scalar divide
  Z += np.exp(-np.real(energy) / (k_B * T))
/tmp/ipykernel_20814/2466187856.py:12: RuntimeWarning: invalid value encountered in scalar multiply
  return -k_B * T * np.log(Z)


M(T=0) =
	nan


/tmp/ipykernel_20814/143915578.py:14: RuntimeWarning: overflow encountered in exp
  Z += np.exp(-np.real(energy) / (k_B * T))
/tmp/ipykernel_20814/4038938142.py:33: RuntimeWarning: invalid value encountered in scalar subtract
  return (F_1 - F_2) / delta_epsilon


M(T=0.01) =
	nan
M(T=0.1) =
	8.306852834669982
M(T=0.5) =
	5.6690480009491395
M(T=1) =
	3.5942018937280196
M(T=10) =
	0.07012207447004926
